In [1]:
import plotly.express as px
import re
import plotly.io as pio
pio.renderers.default = "browser"
from collections import OrderedDict
import argparse
from scipy.stats import gmean
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
from util import to_matrix

In [2]:
time_str = '20220720141500'
bench = ['blackscholes',\
        'bodytrack', \
#          'canneal', \
#          'dedup', \
         'fluidanimate', \
         'raytrace' , \
         'swaptions', \
        ]
sim_str = 'simlarge'
config = [5]
outdir_hist = [f'{time_str}/{b}_{sim_str}_64_{c}/stats.txt' for b in bench for c in config ]
outdir_hist

['20220720141500/blackscholes_simlarge_64_5/stats.txt',
 '20220720141500/bodytrack_simlarge_64_5/stats.txt',
 '20220720141500/fluidanimate_simlarge_64_5/stats.txt',
 '20220720141500/raytrace_simlarge_64_5/stats.txt',
 '20220720141500/swaptions_simlarge_64_5/stats.txt']

In [3]:
pattern_map = {
    'l1w': 'system.ruby.l1_cntrl\d+.cache.writeLinearHist::0-31\s+',
    'l1r': 'system.ruby.l1_cntrl\d+.cache.readLinearHist::0-31\s+',
    'l2r': 'system.ruby.l2_cntrl\d+.L2cache.readLinearHist::0-31\s+',
    'l2w': 'system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::0-31\s+',
    'l2_t': 'system.ruby.l2_cntrl\d+.L2cache.m_demand_accesses\s+\d+',
    'l2_inf': 'system.ruby.l2_cntrl\d+.L2cache.infiniteSD\s+\d+',
    'l1_t': 'system.ruby.l1_cntrl\d+.cache.m_demand_misses\s+\d+',
    'l1_inf': 'system.ruby.l1_cntrl\d+.cache.infiniteSD\s+\d+',
    'l1w1': 'system.ruby.l1_cntrl\d+.cache.writeLinearHist::32-63\s+',
    'l1r1': 'system.ruby.l1_cntrl\d+.cache.readLinearHist::32-63\s+',
    'l2r1': 'system.ruby.l2_cntrl\d+.L2cache.readLinearHist::32-63\s+',
    'l2w1': 'system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::32-63\s+',
}

prune_map = {
    'l2r': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.readLinearHist::0-31\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l2w': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::0-31\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l1r': lambda line: int(re.sub('system.ruby.l1_cntrl\d+.cache.readLinearHist::0-31\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l1w': lambda line: int(re.sub('system.ruby.l1_cntrl\d+.cache.writeLinearHist::0-31\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l2_t': lambda line: int(line.replace(' ','').split('demand_accesses')[-1].split('#')[0]),
    'l2_inf': lambda line: int(line.replace(' ','').split('infiniteSD')[-1].split('#')[0].split('(')[0]),
    'l1_t': lambda line: int(line.replace(' ','').split('demand_misses')[-1].split('#')[0]),
    'l1_inf': lambda line: int(line.replace(' ','').split('infiniteSD')[-1].split('#')[0].split('(')[0]),
    'l2r1': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.readLinearHist::32-63\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l2w1': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::32-63\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l1r1': lambda line: int(re.sub('system.ruby.l1_cntrl\d+.cache.readLinearHist::32-63\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l1w1': lambda line: int(re.sub('system.ruby.l1_cntrl\d+.cache.writeLinearHist::32-63\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
}

In [4]:
def get_list(file, metric):
    pattern = pattern_map[metric]

    list_ = []
    with open(file, 'r') as searchfile:
        for line in searchfile:
            if re.match(pattern, line):
                prune_func = prune_map[metric]
                res = prune_func(line)
                list_.append(res)
    # print(list_)
    return list_

In [5]:
metric = 'l1_t'
l1_t = [get_list(o, metric) for o in outdir_hist if '_5' in o]
# print(f'sd_t ={sd_t}')
l1_t_sum = [sum(arr) for arr in l1_t]
l1_t_sum



[688660, 108507794, 25288051, 9000018, 14458914]

In [6]:
metric = 'l1r'
l1r = [get_list(o, metric) for o in outdir_hist if '_5' in o]
assert len(l1r[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
l1r_sum = [sum(arr) for arr in l1r]
l1r_mean = [float(i/t*100) for i,t in zip(l1r_sum, l1_t_sum)]
l1r_mean

[4.484651351900793,
 0.3731280353925544,
 1.8519537152151426,
 0.2683994632010736,
 1.4041441839961148]

In [7]:
metric = 'l1w'
l1w = [get_list(o, metric) for o in outdir_hist if '_5' in o]
assert len(l1w[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
l1w_sum = [sum(arr) for arr in l1w]
l1w_mean = [float(i/t*100) for i,t in zip(l1w_sum, l1_t_sum)]
l1w_mean

[3.564894142247263,
 0.4840518645139906,
 7.010480957982883,
 1.230464205738255,
 0.4314570236741155]

In [8]:
l1_mean0 = [r+w for r,w in zip(l1r_mean, l1w_mean)]
l1_mean0

[8.049545494148056,
 0.857179899906545,
 8.862434673198026,
 1.4988636689393287,
 1.8356012076702304]

In [9]:
metric = 'l1r1'
l1r1 = [get_list(o, metric) for o in outdir_hist if '_5' in o]
assert len(l1r1[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
l1r1_sum = [sum(arr) for arr in l1r1]
l1r1_mean = [float(i/t*100) for i,t in zip(l1r1_sum, l1_t_sum)]


metric = 'l1w1'
l1w1 = [get_list(o, metric) for o in outdir_hist if '_5' in o]
assert len(l1w1[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
l1w1_sum = [sum(arr) for arr in l1w1]
l1w1_mean = [float(i/t*100) for i,t in zip(l1w1_sum, l1_t_sum)]

l1_mean1 = [r1+w1+r2+w2 for r1,w1,r2,w2 in zip(l1r_mean, l1w_mean,l1r1_mean, l1w1_mean)]
l1_mean1

[16.333894810211135,
 1.1428948597001243,
 9.151768952063565,
 1.6335633995398673,
 5.192858882762564]

In [10]:
metric = 'l1_inf'
l1inf = [get_list(o, metric) for o in outdir_hist if '_5' in o]
assert len(l1inf[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
l1inf_sum = [sum(arr) for arr in l1inf]
l1inf_mean = [float(i/t*100) for i,t in zip(l1inf_sum, l1_t_sum)]
l1inf_mean

[44.31983852699445,
 2.054354731421413,
 8.919857050272478,
 71.03092460481746,
 4.4746583318774835]

## l1 low stack dist, inf stack dist

In [11]:
l1r

[[273,
  555,
  514,
  461,
  219,
  415,
  532,
  513,
  330,
  463,
  408,
  451,
  494,
  522,
  445,
  405,
  330,
  231,
  498,
  531,
  492,
  432,
  517,
  674,
  523,
  528,
  438,
  440,
  303,
  506,
  508,
  538,
  505,
  528,
  469,
  387,
  439,
  396,
  521,
  420,
  481,
  570,
  568,
  377,
  551,
  666,
  526,
  523,
  482,
  583,
  378,
  446,
  318,
  575,
  582,
  592,
  578,
  543,
  558,
  346,
  571,
  758,
  625,
  533],
 [2396,
  6028,
  5981,
  6169,
  6879,
  6215,
  6022,
  6334,
  6061,
  6316,
  6117,
  6244,
  8983,
  5949,
  6511,
  5921,
  6071,
  5909,
  6177,
  5936,
  6131,
  5748,
  6484,
  6237,
  6272,
  6222,
  5936,
  6037,
  5886,
  6375,
  6384,
  6441,
  6000,
  5945,
  6179,
  6385,
  6237,
  6353,
  6139,
  6115,
  6204,
  6427,
  6568,
  5976,
  6279,
  9149,
  6455,
  6250,
  6201,
  6325,
  6154,
  5935,
  5979,
  5943,
  6208,
  6111,
  7341,
  5881,
  6115,
  8867,
  8904,
  6310,
  6791,
  6775],
 [10166,
  9765,
  7108,
  9081,
  923

In [12]:
l1 = [r+r1+w+w1 for r,r1,w,w1 in zip(l1r, l1r1, l1w,l1w1)]
l1_good_p = [[float(n/t*100) for n,t in zip(num,tot) ] for num,tot in zip(l1, l1_t)]
l1_inf_p = [[float(n/t*100) for n,t in zip(num,tot) ] for num,tot in zip(l1inf, l1_t)]

In [13]:
metric = 'l2_t'
l2_t = [get_list(o, metric) for o in outdir_hist if '_5' in o]
# print(f'sd_t ={sd_t}')
l2_t_sum = [sum(arr) for arr in l2_t]
l2_t_sum

[688660, 108507794, 25288051, 9000018, 14458914]

In [14]:
metric = 'l2r'
l2r = [get_list(o, metric) for o in outdir_hist if '_5' in o]
assert len(l2r[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
l2r_sum = [sum(arr) for arr in l2r]
l2r_mean = [float(i/t*100) for i,t in zip(l2r_sum, l2_t_sum)]
l2r_mean

[23.987744315046612,
 59.523275351077544,
 11.149799563438084,
 9.793558190661395,
 61.60140381220886]

In [15]:
metric = 'l2w'
l2w = [get_list(o, metric) for o in outdir_hist if '_5' in o]
assert len(l2w[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
l2w_sum = [sum(arr) for arr in l2w]
l2w_mean = [float(i/t*100) for i,t in zip(l2w_sum, l2_t_sum)]
l2w_mean

[3.1403014549995647,
 0.7281523021286379,
 7.662662496212144,
 1.382486123916641,
 1.167224592386399]

In [16]:
l2_mean0 = [r+w for r,w in zip(l2r_mean, l2w_mean)]
l2_mean0

[27.128045770046178,
 60.25142765320618,
 18.81246205965023,
 11.176044314578036,
 62.768628404595255]

In [17]:
metric = 'l2r1'
l2r1 = [get_list(o, metric) for o in outdir_hist if '_5' in o]
assert len(l2r1[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
l2r1_sum = [sum(arr) for arr in l2r1]
l2r1_mean = [float(i/t*100) for i,t in zip(l2r1_sum, l2_t_sum)]


metric = 'l2w1'
l2w1 = [get_list(o, metric) for o in outdir_hist if '_5' in o]
assert len(l2w1[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
l2w1_sum = [sum(arr) for arr in l2w1]
l2w1_mean = [float(i/t*100) for i,t in zip(l2w1_sum, l2_t_sum)]

l2_mean1 = [r1+w1+r2+w2 for r1,w1,r2,w2 in zip(l2r_mean, l2w_mean,l2r1_mean, l2w1_mean)]
l2_mean1

[47.34295588534255,
 86.32266913471672,
 22.771909151875725,
 15.289802753727823,
 69.71596898632913]

In [18]:
metric = 'l2_inf'
l2inf = [get_list(o, metric) for o in outdir_hist if '_5' in o]
assert len(l2inf[0]) == 64
# sd_rd_0_pc = [[float(ii/tt)*100 for ii, tt in zip(i, t)] for i,t in zip(sd_rd_0, sd_t)]
# print(f'sd_rd_0 percent ={sd_rd_0_pc}')
l2inf_sum = [sum(arr) for arr in l2inf]
l2inf_mean = [float(i/t*100) for i,t in zip(l2inf_sum, l2_t_sum)]
l2inf_mean

[21.615310893619494,
 0.33525702310379657,
 5.2705920278316425,
 30.173839652320694,
 2.785769387659405]

## l2 low stack dist, inf stack dist

In [19]:
l2 = [r+r1+w+w1 for r,r1,w,w1 in zip(l2r, l2r1, l2w,l2w1)]
l2_good_p = [[float(n/t*100) for n,t in zip(num,tot) ] for num,tot in zip(l2, l2_t)]
l2_inf_p = [[float(n/t*100) for n,t in zip(num,tot) ] for num,tot in zip(l2inf, l2_t)]

# plot

In [20]:
l1_std_sd = []
l2_std_sd = []
for l1_arr, l2_arr, l1_anno_arr, l2_anno_arr, \
    name,  \
    in zip(l1_good_p, l2_good_p, l1_good_p, l2_good_p, \
        bench, ):
    
    l1_matrix = to_matrix(l1_arr, 8)
    l2_matrix = to_matrix(l2_arr, 8)
    
    l1_anno_matrix = to_matrix(l1_anno_arr, 8)
    l2_anno_matrix = to_matrix(l2_anno_arr, 8)
    
    # rouding
#     l1_matrix = [ [round(elem, 2) for elem in my_list] for my_list in l1_matrix ]
#     l2_matrix = [ [round(elem, 2) for elem in my_list] for my_list in l2_matrix ]
    l1_anno_matrix = [ [round(elem, 2) for elem in my_list] for my_list in l1_anno_matrix ]
    l2_anno_matrix = [ [round(elem, 2) for elem in my_list] for my_list in l2_anno_matrix ]

    
    # l1 heatmap
    fig = px.imshow(l1_matrix,
                labels=dict(x="", y="", color= f"l1 low stack distance (<64) line percentage"),
               )

    l1_std_ = np.std(l1_arr)
    l1_std_sd.append(l1_std_)
    
    fig.update_layout(
        title=f"{name}: std={l1_std_}",
    )
    fig.update_traces(text=l1_anno_matrix, texttemplate="%{text}")
    fig.show()
    
    # l2 heatmap
#     print(l2_matrix)
    fig = px.imshow(l2_matrix,
                labels=dict(x="", y="", color=f"l2 low stack distance (<64) line percentage"),
               )
    l2_std_ = np.std(l2_arr)
    l2_std_sd.append(l2_std_)
    fig.update_layout(
        title=f"{name}: std={l2_std_}",
    )
    fig.update_traces(text=l2_anno_matrix, texttemplate="%{text}")
    fig.show()